In [1]:
from consensusModel import clsf
import pandas as pd, numpy as np
from tqdm import tqdm

In [2]:
nCV=100
folds = 5

In [3]:
X = pd.read_csv("data/trainMatrix.csv")

In [4]:
Y = pd.read_csv("data/curedData.csv", usecols=["label"]).squeeze()

## Permutation test against chance

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
import random

In [6]:
Yshuff = pd.read_csv("data/Yshuffled.csv")

In [7]:
Yshuff.columns = Yshuff.columns.astype(int)

In [12]:
allScoresShuffledY

0                                            1             \
           RF       SVM       XGB  KNN       ADA        RF       SVM   
0    0.511667  0.823388  0.701561  0.6  0.506765    0.3775  0.843076   
1        0.98  0.827924  0.906985  1.0  0.681088  0.248333  0.828265   
2    0.751964   0.83822   0.64394  1.0  0.678315  0.299667  0.860353   
3    0.751964   0.83822   0.64394  1.0  0.678315  0.886167  0.833111   
4        0.98   0.82833  0.913411  1.0   0.67745  0.976667  0.828299   
..        ...       ...       ...  ...       ...       ...       ...   
728  0.560127  0.813022  0.393435  1.0  0.506464  0.867597  0.857084   
729  0.733905  0.826331   0.58005  0.6  0.506464  0.940452   0.83619   
730      0.55  0.804595  0.376386  0.8  0.502339      0.99  0.829162   
731      0.87  0.831939    0.7127  0.8   0.50758      0.98  0.828119   
732      0.92  0.831915  0.792275  0.8   0.50758      0.97  0.830874   

                              ...        98                           \
          XGB  KNN       ADA  ...        RF       SVM       XGB  KNN   
0    0.724537  0.6  0.507639  ...  0.646667  0.832795   0.76108  0.8   
1    0.605633  0.6  0.507334  ...      0.99  0.831958  0.989084  0.8   
2    0.177951  0.6  0.499046  ...  0.674643  0.843226  0.912278  0.8   
3    0.917881  0.8  0.502741  ...  0.674643  0.843226  0.912278  0.8   
4    0.930282  0.8  0.507465  ...  0.920833   0.83231   0.91459  0.8   
..        ...  ...       ...  ...       ...       ...       ...  ...   
728  0.972228  0.8  0.507465  ...  0.714167  0.827994  0.659831  0.8   
729  0.951283  0.8  0.507334  ...  0.810167  0.828035  0.881714  1.0   
730   0.96636  1.0  0.509027  ...  0.608476  0.831569  0.797174  0.6   
731  0.971579  1.0  0.509027  ...  0.912333   0.82159  0.902045  0.8   
732  0.865013  0.8  0.507334  ...  0.727008   0.83205  0.927026  0.6   

                     99                                     
          ADA        RF       SVM       XGB  KNN       ADA  
0     0.50276  0.961667  0.835149  0.920987  1.0  0.509919  
1    0.510817     0.985  0.820471  0.972674  1.0  0.681614  
2    0.507904  0.798333   0.83566  0.656205  1.0  0.501111  
3    0.507904    0.8985  0.836644  0.829601  1.0  0.679272  
4    0.503223     0.985  0.820471  0.972674  1.0  0.681614  
..        ...       ...       ...       ...  ...       ...  
728  0.504309   0.96531  0.804274   0.98283  1.0  0.691237  
729  0.508457  0.822992  0.837548  0.899078  1.0  0.510088  
730  0.508457      0.94  0.836655  0.754723  1.0  0.507293  
731  0.520907      0.97  0.827367  0.914669  1.0  0.509919  
732  0.680629       1.0  0.826841   0.94847  1.0  0.510088  

[733 rows x 500 columns]

## Results

In [8]:
from utilsMetric import metrics, getTableFromStats

In [9]:
shuffledScores = pd.read_csv("results/YshuffledCVscores.csv", index_col=0, header=[0,1])
Yshuff = pd.read_csv("data/Yshuffled.csv")

In [9]:
Yshuff

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1,1,0,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,0,...,1,0,1,1,1,1,1,0,1,1
2,1,1,1,0,1,0,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
3,1,0,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,0,1,0,1
4,1,0,1,0,1,1,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,1,1,1,1,1,1,1,0,1,1,...,0,1,1,1,1,0,1,1,1,0
729,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
730,1,1,0,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
731,0,1,0,1,0,1,1,1,1,1,...,1,0,1,1,0,1,1,1,0,1


In [10]:
shuffledConsensusScores =pd.concat( 
            [shuffledScores[c].mean(axis=1) for c in shuffledScores.columns.levels[0] ], 
            axis=1, names = range(nCV)
                            )

In [11]:
thrs = [float(f"0.{x}") for x in range(1,10)]
results = []

# Storing performance from CV predictions on each cutoff classification score from 0.1 to 0.9
for cutoff in thrs:
    shuffledPreds = shuffledConsensusScores.applymap(lambda x : 1 if x>=cutoff else 0)
    metricsShuffled = pd.DataFrame(columns=shuffledPreds.columns,
                            index=[*metrics] 
                               )

    for metrName, metrFun in metrics.items():
        metricsShuffled.loc[metrName] = [ metrFun(Ycv,preds, scores)
                            for Ycv, preds, scores in zip(Yshuff.values.T, shuffledPreds.values.T, shuffledConsensusScores.values.T)  ]

    results.append(getTableFromStats(metricsShuffled.T.astype(float).describe()))

/home/toxi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/toxi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/toxi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/toxi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWar

In [12]:
pd.concat(results,axis=1)

,median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)",median,"(1st quartile, 3rd quartile)"
spec,0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.016129032258064516,"0.008064516129032258, 0.024193548387096774",0.06451612903225806,"0.04838709677419355, 0.08064516129032258",0.18951612903225806,"0.16129032258064516, 0.21774193548387097",0.5161290322580645,"0.4838709677419355, 0.5564516129032258",0.9838709677419355,"0.9758064516129032, 0.9919354838709677"
sens,1.0,"1.0, 1.0",1.0,"1.0, 1.0",1.0,"1.0, 1.0",0.9991789819376027,"0.9967159277504105, 1.0",0.9852216748768473,"0.9802955665024631, 0.9885057471264368",0.9359605911330049,"0.9277504105090312, 0.9425287356321839",0.8111658456486043,"0.7976190476190477, 0.8214285714285714",0.4860426929392447,"0.4663382594417077, 0.49958949096880134",0.0180623973727422,"0.013136288998357963, 0.02504105090311987"
balacc,0.5,"0.5, 0.5",0.5,"0.5, 0.5",0.5,"0.5, 0.5",0.5,"0.49917898193760263, 0.5",0.49903331744266116,"0.49587670162614544, 0.5033254542083797",0.49941734202023413,"0.49140745007680486, 0.5051644684570157",0.50242994862016,"0.4859086153927644, 0.5168805286296944",0.504214338683193,"0.48607910906298, 0.5190936357857937",0.5001820806186769,"0.4960405741829546, 0.503429736744531"
mcc,0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"-0.01667810315848988, 0.0",-0.00619139478335208,"-0.02953719208865079, 0.022648782228838843",-0.001766026755166335,"-0.02712670890275388, 0.016226970822670318",0.004727647959401989,"-0.027831295324376174, 0.0320423190191768",0.006333912404005427,"-0.020892122849251883, 0.028638773865761857",0.0009596106413684142,"-0.022221494686734993, 0.02458923609821219"
ppv,0.8308321964529332,"0.8308321964529332, 0.8308321964529332",0.8308321964529332,"0.8308321964529332, 0.8308321964529332",0.8308321964529332,"0.8308321964529332, 0.8308321964529332",0.8308321964529332,"0.8306010928961749, 0.8308321964529332",0.8305555555555556,"0.8296627195519163, 0.831779214559387",0.8306569343065694,"0.8282718009792246, 0.8323699421965318",0.8316666666666667,"0.8260280633509308, 0.836593851132686",0.8331255195344971,"0.8226984126984127, 0.8414755227751662",0.8333333333333334,"0.763184584178499, 0.9022727272727273"
npv,0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.0,"0.0, 0.0",0.14835164835164835,"0.08333333333333333, 0.25",0.16666666666666666,"0.129740980573543, 0.19102186711522287",0.17293233082706766,"0.14658536585365853, 0.19405013591060105",0.1714985994397759,"0.1616550521484075, 0.1796292350308971",0.16922010028806145,"0.16804407713498623, 0.1701534170153417"
PRcurve,0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426",0.8330350089010461,"0.8229418912256465, 0.8414454461561426"
ROCcurve,0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059",0.5045950527040628,"0.48547327718629163, 0.5213199851687059"
